<a href="https://colab.research.google.com/github/Priyansh-Kedia/AmazonChallenge/blob/master/AmazonChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Add all imports here

from nltk.corpus import stopwords
import nltk
import string
import re
from collections import Counter

In [ ]:
# Download dataset

!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip


--2021-07-30 09:43:25--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.124.194
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.124.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip.4’

dataset52a7b21.zip. 100%[===================>]   1012M  17.8MB/s    in 59s     

2021-07-30 09:44:25 (17.2 MB/s) - ‘dataset52a7b21.zip.4’ saved [1061576029/1061576029]



In [ ]:
# Unzip the dataset

!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
replace dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/train.csv       

replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [{ENTER}]
replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/sample_submission.csv  
replace dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/test.csv        y

replace dataset/.~lock.train.csv#? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: dataset/.~lock.train.csv#  


In [ ]:
# Download nltk stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Import the dataset in pandas

import pandas as pd
import csv
df = pd.read_csv("dataset/train.csv", low_memory=True, escapechar="\\", quoting=csv.QUOTE_NONE)

In [ ]:
# Add the column variables here

TITLE = "TITLE"
PRODUCT_ID = "PRODUCT_ID"
DESCRIPTION = "DESCRIPTION"
BULLET_POINTS = "BULLET_POINTS"
BRAND = "BRAND"
BROWSE_NODE_ID = "BROWSE_NODE_ID"

In [116]:
# Load the doc
def load_doc(df):
  titles = ' '.join(df[TITLE].fillna('').astype(str))
  descriptions = ' '.join(df[DESCRIPTION].fillna('').astype(str))
  bullets = ' '.join(df[BULLET_POINTS].fillna('').astype(str))

  return (titles + descriptions + bullets)

In [117]:
# To remove html tags

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>') # replace with re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') for more strictness
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [118]:
def clean_doc(doc):
  doc = cleanhtml(doc)
  tokens = doc.split() # Add delimiter if needed

  punctuations = str.maketrans('','', string.punctuation)

  tokens = [w.translate(punctuations) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  stop_words = set(stopwords.words("english"))
  print("stop", stop_words)
  tokens = [w for w in tokens if not w.casefold() in stop_words]

  tokens = [word for word in tokens if len(word) > 1]

  return tokens


In [119]:
# Create, Store, and Read Hashmap functions

def create_hashmap(dataframe):
    from collections import defaultdict
    Hashmap = defaultdict(list)

    for i, [brand, id] in enumerate(dataframe.values[:, [3, 4]]):
        if int(id) not in Hashmap[str(brand)]:
            Hashmap[str(brand)].append(int(id))

    return Hashmap

def store_hashmap(Hashmap, filename="Hashmap"):
    import pickle
    
    with open(filename + '.pickle', 'wb') as handle:
        pickle.dump(Hashmap, handle)

def read_hashmap(filename="Hashmap"):
    import pickle
    
    with open(filename + '.pickle', 'rb') as handle:
        Hashmap = pickle.load(handle)

    return Hashmap

In [120]:
# Add words to vocab

def add_doc_to_vocab(df):
  doc = load_doc(df)
  print(doc)
  tokens = clean_doc(doc)
  print(tokens)
  vocab.update(tokens)

In [133]:
# Save the vocab

def save_list(lines, filename):
    # convert the lines into single blob of text
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [121]:
# This is just created to test the above functions

tokens = clean_doc('testing this for the use case <p>this should also be tested</p> <br>test this also breaking</br> <h1>and test heading too</h1>')
print(tokens)

stop {'m', 'while', 'under', 'was', 'about', 'during', 'if', 'they', "shouldn't", 'isn', 'me', 'by', 'am', "it's", 'so', 'herself', 'which', "isn't", 'has', 'down', 'each', 'through', 'are', "you're", 'same', "she's", 'below', 'or', 'from', 'all', 'whom', 'out', "wouldn't", 'will', 'him', 'between', 'ourselves', 'what', 'don', 'did', 'after', 'we', 'wasn', 'ain', 'themselves', 'in', 'does', "weren't", 'when', 'didn', "couldn't", 'he', 'their', "hasn't", 'a', 'own', 'more', 'nor', "haven't", 'do', 'who', 'the', "should've", "hadn't", "won't", 'shouldn', "that'll", "you'd", 'few', 'until', 'be', 'up', 'hasn', 'haven', "mustn't", 'why', 'an', 'shan', 'any', 'her', 'needn', 'above', 't', "didn't", 'ours', 'she', 've', 'doesn', 'its', 'those', 'now', 's', 'how', 'and', 'further', 'hers', 'both', 'such', 'couldn', 'no', 'them', 'o', 'of', 'yourself', 'just', 'this', 'here', 'mustn', 'is', 'been', 'only', 'wouldn', 'our', 'himself', 'theirs', 'yours', "shan't", 'mightn', 'into', 'than', 'with

In [122]:
# Get cleaned document from here

loaded = load_doc(df.head(2000))
tokens = clean_doc(loaded)

# print(tokens)

stop {'m', 'while', 'under', 'was', 'about', 'during', 'if', 'they', "shouldn't", 'isn', 'me', 'by', 'am', "it's", 'so', 'herself', 'which', "isn't", 'has', 'down', 'each', 'through', 'are', "you're", 'same', "she's", 'below', 'or', 'from', 'all', 'whom', 'out', "wouldn't", 'will', 'him', 'between', 'ourselves', 'what', 'don', 'did', 'after', 'we', 'wasn', 'ain', 'themselves', 'in', 'does', "weren't", 'when', 'didn', "couldn't", 'he', 'their', "hasn't", 'a', 'own', 'more', 'nor', "haven't", 'do', 'who', 'the', "should've", "hadn't", "won't", 'shouldn', "that'll", "you'd", 'few', 'until', 'be', 'up', 'hasn', 'haven', "mustn't", 'why', 'an', 'shan', 'any', 'her', 'needn', 'above', 't', "didn't", 'ours', 'she', 've', 'doesn', 'its', 'those', 'now', 's', 'how', 'and', 'further', 'hers', 'both', 'such', 'couldn', 'no', 'them', 'o', 'of', 'yourself', 'just', 'this', 'here', 'mustn', 'is', 'been', 'only', 'wouldn', 'our', 'himself', 'theirs', 'yours', "shan't", 'mightn', 'into', 'than', 'with

In [128]:
# create the vocab

vocab = Counter()
add_doc_to_vocab(df.head(100))

# print(vocab.most_common(50))

Pete The Cat Bedtime Blues Doll, 14.5 Inch The New Yorker NYHM014 Refrigerator Magnet, 2 x 3.5 The Ultimate Self-Sufficiency Handbook: A Complete Guide to Baking, Crafts, Gardening, Preserving Your Harvest, Raising Animals and More Amway Nutrilite Kids Chewable Iron Tablets (100) Teacher Planner Company A4 6 Lesson Academic Teacher Planner - Leatherette Pink Men'S Full Sleeve Raglan T-Shirts Denim T-Shirt M Size Only Glance Women's Wallet (Black) (LW-21) Wild Animals Hungry Brain Educational Flash Cards for Enhancing Early Learning of Your Kids(3 Month to 6 Year) Hindi Barakhadi Educational Flash Cards for Kids Enhance Early Learning of Your Kids (3 Months to 6 Years) Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Steel Sprayer for Fertilizer Slo-Niacin Polygel Controlled-Release Niacin, 750 mg, 100 Tablets White : Braun KF400-WH Aromaster 10-Cup Coffeemaker, White Yamaha PCMSKIT Portable Electronic Keyboard Adapter with PC Joystick to MIDI Cable Microsoft Sidewinder Gamepad Pro Jiffy Steam

In [135]:
# Save vocab according to occurance

min_occurane = 10
tokens = [k for k,c in vocab.items() if c >= min_occurane]

save_list(tokens, "vocab.txt")